In [53]:
import numpy as np
from sklearn.cluster import KMeans
from skimage.io import imread
import pylab
from skimage import img_as_float
import numpy.lib.recfunctions as rfn
import pandas as pd

image = img_as_float(imread('parrots.jpg'))

#%matplotlib inline
#pylab.imshow(image)

X = np.empty(shape = [image[:,:,0].size, 3])

width = image.shape[0]
height = image.shape[1]

for i in range(width*height):
    X[i] = image[i%width, i//width]

for number in range(2,21):
    kmeans = KMeans(n_clusters = number, init='k-means++', random_state=241).fit(X)

    L = kmeans.labels_
    XL = np.hstack((X, L.reshape(width*height, 1)))

    clusters_avg = np.empty([kmeans.n_clusters, 4])
    clusters_median = np.empty([kmeans.n_clusters, 4])

    for i in range(kmeans.n_clusters):
            clusters_avg[i] = np.average(XL[XL[:, 3] == i], axis = 0)
            clusters_median[i] = np.median(XL[XL[:, 3] == i], axis = 0)

    XL_avg = np.empty(XL.shape)

    for i in range(XL.shape[0]):
        for j in range(kmeans.n_clusters):
            if XL[i][3] == j:
                XL_avg[i] = clusters_avg[clusters_avg[:,3]==j]

    XL_median = np.empty(XL.shape)

    for i in range(XL.shape[0]):
        for j in range(kmeans.n_clusters):
            if XL[i][3] == j:
                XL_median[i] = clusters_median[clusters_median[:,3]==j]

    XL_avg_img = np.empty(image.shape)
    XL_median_img = np.empty(image.shape)

    for i in range(XL_avg.shape[0]):
        XL_avg_img[i%width][i//width] = XL_avg[i, :3]
        XL_median_img[i%width][i//width] = XL_median[i, :3]

    XL_df = pd.DataFrame(XL, columns = ['R', 'G', 'B', 'C'])
    clusters_avg_df = pd.DataFrame(clusters_avg, columns = ['R1', 'G1', 'B1', 'C'])
    clusters_med_df = pd.DataFrame(clusters_median, columns = ['R1', 'G1', 'B1', 'C'])

    XLC_avg = pd.merge(XL_df, clusters_avg_df, how = 'left', on='C')
    XLC_med = pd.merge(XL_df, clusters_med_df, how = 'left', on='C')

    MSE_avg = 0
    MSE_med = 0

    for i in range(width):
        for j in range(height):
            MSE_avg += (XLC_avg['R'][width*i + j] - XLC_avg['R1'][width*i + j])**2 + \
                            (XLC_avg['G'][width*i + j] - XLC_avg['G1'][width*i + j])**2 + \
                                (XLC_avg['B'][width*i + j] - XLC_avg['B1'][width*i + j])**2
            MSE_med += (XLC_med['R'][width*i + j] - XLC_med['R1'][width*i + j])**2 + \
                            (XLC_med['G'][width*i + j] - XLC_med['G1'][width*i + j])**2 + \
                                (XLC_med['B'][width*i + j] - XLC_med['B1'][width*i + j])**2

    MSE_avg = MSE_avg/(width*height*3)
    MSE_med = MSE_med/(width*height*3)

    PSNR_avg = 10*np.log10(1/MSE_avg)
    PSNR_med = 10*np.log10(1/MSE_med)

    print(kmeans.n_clusters, PSNR_avg, PSNR_med)
    
print('The End')

2 11.829538714229892 11.453855798729606
3 13.08834030117 12.797802016867347
4 14.121782238180572 13.845192504792374
5 15.58334815622835 15.54932240404887
6 16.67977217769305 15.912275854988554
7 17.670754403729823 17.350141429845625
8 18.253901877412584 17.90156383646155
9 19.24929693009147 19.07337807321442
10 19.81218148306837 19.666271195258176
11 20.237442041728197 20.081578490082475
12 20.71072245715765 20.564975602726577
13 21.119847369948147 20.94358723066424
14 21.300969148608274 21.153589677464556
15 21.55210933371098 21.398261758031012
16 21.75858429353393 21.611285234605223
17 21.98097168904585 21.822912701849887
18 22.289406527382482 22.14660477946253
19 22.37539554711705 22.229379470628846
20 22.684490777887074 22.550519926066848
The End
